## Lesson 13

### Back Progapation

#### The forward and backward passes

In [4]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path("data")
path_data.mkdir(exist_ok=True)
path_gz = path_data/"mnist.pkl.gz"

from urllib.request import urlretrieve

if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

##### Basic Architecture

In [5]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [6]:
# num hidden
# the number of line thingies we want to add up
nh = 50

In [7]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [8]:
def lin(x, w, b): return x@w + b

In [9]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [10]:
def relu(x): return x.clamp_min(0.)

In [11]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [12]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [13]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

##### Loss function: MSE

* Not a suitable loss function for multi-clas classification but using it to keep things simple

In [14]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [15]:
(res-y_valid).shape

torch.Size([10000, 10000])

In [16]:
res[:,0].shape

torch.Size([10000])

In [17]:
res.squeeze().shape

torch.Size([10000])

In [18]:
(res.squeeze() - y_valid).shape

torch.Size([10000])

In [19]:
y_train,y_valid = y_train.float(), y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [20]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [21]:
mse(preds, y_train)

tensor(4308.76)

#### Gradient and backward pass

In [22]:
from sympy import symbols,diff
x,y = symbols("x y")
diff(x**2, x)

2*x

##### Python Debugger
Understanding how to use the Python debugger is one of the most powerful things you can do to write better code.

Note: this does not seem to work inside vscode, might have to switch to the web version to get it to work

import pdb; pdb.set_trace()

This will stop the code at this line and allow you to inspect the code.

hit h to see the list of options you can do
* You can use p to print things from inside the function
* c to continue, the code will continue until it hits set_trace() again
* w tells you what called this function
* n goes to the next line
* can test out different ways of executing the code from within the function

Suggested: look up a Python pdb tutorial

In [28]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.T@out.g
    b.g = out.g.sum(0)

In [29]:

def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [30]:
forward_and_backward(x_train, y_train)

In [33]:
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

In [34]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [35]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [36]:
loss = forward(xt2, y_train)
loss.backward()

In [37]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

#### Refector model

##### Layers as classes

In [38]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [39]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)